In [130]:
def delete_column(data, column):
    ''' data: 데이터 프레임 이름, column: 변수 이름(string type으로 입력)
        원하는 컬럼 제거 후 리턴
    '''
    result = data.drop(columns=[column])
    return result

def to_category(data, column):
    ''' 선택한 컬럼의 변수 타입을 카테고리 타입으로 변경
        data = 데이터프레임, column = 컬럼
    '''
    result = data.astype({column:'category'})
    return result

def combine_category(data, column, before, after):
    ''' 한 컬럼 내의 특정 카테고리들 합치기
        data = 데이터셋(pd.DataFrame), column = 선택할 컬럼(string),
        before = 합칠 카테고리들(list),  after = 합치고 난 후의 클래스 이름(string)
    '''
    num_of_classes = len(before)
    l_after = []
    
    for i in range(num_of_classes):
        l_after.append(after)
        
    answer = data[column].replace(before, l_after)
    data[column] = answer
    
    return data

# 연세대학교 마일리지 예측모델

In [1]:
%matplotlib inline


import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import font_manager, rc
from matplotlib import style

import seaborn as sns
import random

In [22]:
import pandas_profiling
# 컬럼 갯수 무제한으로
pd.set_option('display.max_columns', None)

### 데이터 가져오기

In [29]:
# 인덱스가 없음으로 'index_col=False' 추가
data1 = pd.read_csv('/Users/appdesign490/Desktop/YBIGTA/ScienceTeam/MiniP/final/yonseimileage_2019_1.csv', encoding='euc-kr', index_col=False)
data1.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (5,10,13,14,15,18,19,20,21,22,24,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
0,학부,교양기초(2019~),채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,1,2,Y (Y),6,N,Y,0.4603,0.8421,3,O
1,학부,교양기초(2019~),채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,2,2,Y (Y),5,Y,Y,0.873,0.8333,4,O
2,학부,교양기초(2019~),채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,3,1,Y (Y),6,N,Y,0.5238,0.6315,3,O
3,학부,교양기초(2019~),채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,4,1,Y (Y),5,N,Y,0.7857,0.5,4,O
4,학부,교양기초(2019~),채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,1,20,Y (Y),5,N,Y,0.2777,0.8947,2,O


In [30]:
data1.describe()

,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
count,63550,63550,63550,63550,63550,63550,63549,62309,62309,63550,63549,63549,63533,63526,63526,63526,63524,63491,62966.0,62966.0,62966,62966,62966.0,62966,62966.0,62966,62964,62962,62962.0,62962.0,62962
unique,2,33,182,2417,1712,13,1489,674,626,129,309,116,50,416,164,108,11,9,48.0,74.0,1576,131,74.0,4,14.0,3,3,1573,180.0,11.0,3
top,학부,공과대학,경영학전공,STA1001-12-01,통계학입문,3,김미정,"화4,목5,6",수영장(신촌),40,24,0 (N),0,0,0,0,O,36,1.0,36.0,1,1,36.0,N (N),6.0,N,Y,0.5000,1.0,4.0,O
freq,63548,7670,2463,152,1237,26518,439,2486,849,4861,1161,37017,61469,25994,25149,30152,38376,53818,30829.0,20424.0,865,2026,6348.0,36554,27748.0,55445,60130,1737,10421.0,12464.0,57509


In [31]:
data2 = pd.read_csv('/Users/appdesign490/Desktop/YBIGTA/ScienceTeam/MiniP/final/yonseimileage_2019_2.csv', encoding='euc-kr',index_col=False)
data2.head()

,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
0,학부,교양기초(2019~),채플,YCA1002-01-00,채플(2),0,곽호철,화2,I종301,50,1,20 (N),2,20,25,3,X,36,1,1,1,1,1,Y (Y),6,N,Y,0.5238,0.6315,3,O
1,학부,교양기초(2019~),채플,YCA1002-02-00,채플(2),0,곽호철,화3,I종301,50,2,20 (N),2,20,25,3,X,36,2,3,2.5,1,3,Y (Y),6,N,Y,0.5873,0.8333,3,O
2,학부,교양기초(2019~),채플,YCA1002-02-00,채플(2),0,곽호철,화3,I종301,50,2,20 (N),2,20,25,3,X,36,2,3,2.5,2,2,Y (Y),6,N,Y,0.8095,0.6666,4,O
3,학부,교양기초(2019~),채플,YCA1006-01-00,채플(B),0.5,곽호철,화2,I종301,646,19,500 (N),600,40,6,0,X,36,1,10,2,1,10,N (N),6,N,Y,0.9166,1,3,O
4,학부,교양기초(2019~),채플,YCA1006-01-00,채플(B),0.5,곽호철,화2,I종301,646,19,500 (N),600,40,6,0,X,36,1,10,2,2,4,Y (Y),6,N,Y,0.4166,0.8157,2,O


In [32]:
data2.describe()

,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
count,61378,61378,61378,61378,61378,61378,61358,60038,60038,61378,61378,61376,61354,61349,61349,61349,61349,61318,60816.0,60816.0,60816,60816,60816.0,60816,60816.0,60816,60799,60796.0,60796.0,60796.0,60796
unique,2,33,182,2400,1722,14,1611,651,603,133,307,124,48,417,192,125,9,8,44.0,72.0,1524,210,73.0,4,14.0,3,3,1580.0,188.0,11.0,3
top,학부,대학교양(2019~),경영학전공,YCD1602-01-00,통계학입문,3,홍석민,"화4,목5,6",골프장,40,35,0 (N),0,0,0,0,O,36,1.0,36.0,1,1,36.0,N (N),6.0,N,Y,1.0,1.0,4.0,O
freq,61376,7476,2590,183,825,24734,305,2794,907,5195,1193,37337,59698,23722,23198,29001,36304,48445,31168.0,18792.0,605,1980,6007.0,35819,27237.0,47092,58418,1725.0,8666.0,14285.0,55377


### 2019년 1학기, 2학기 데이터 병합

In [33]:
test = [data1, data2]

In [34]:
result = pd.concat(test)

In [99]:
result

,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
0,학부,교양기초(2019~),채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,1,2,Y (Y),6,N,Y,0.4603,0.8421,3,O
1,학부,교양기초(2019~),채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,2,2,Y (Y),5,Y,Y,0.873,0.8333,4,O
2,학부,교양기초(2019~),채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,3,1,Y (Y),6,N,Y,0.5238,0.6315,3,O
3,학부,교양기초(2019~),채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,4,1,Y (Y),5,N,Y,0.7857,0.5,4,O
4,학부,교양기초(2019~),채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,1,20,Y (Y),5,N,Y,0.2777,0.8947,2,O
5,학부,교양기초(2019~),채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,2,15,Y (Y),4,Y,Y,0.8769,0.8611,4,O
6,학부,교양기초(2019~),채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,3,5,N (N),6,N,Y,0.3214,0.9583,2,O
7,학부,교양기초(2019~),채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,4,4,Y (Y),6,N,Y,0.2698,0.8421,2,O
8,학부,교양기초(2019~),채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,5,4,Y (Y),5,N,Y,0.3412,1,2,O
9,학부,교양기초(2019~),채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,6,3,Y (Y),6,N,Y,0.5515,1,3,O


In [35]:
result.describe()

,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
count,124928,124928,124928,124928,124928,124928,124907,122347,122347,124928,124927,124925,124887,124875,124875,124875,124873,124809,123782.0,123782.0,123782,123782,123782.0,123782,123782.0,123782,123763,123758.0,123758.0,123758.0,123758
unique,2,33,187,3988,2943,14,2127,946,794,159,384,156,80,651,215,144,13,9,55.0,74.0,2469,210,74.0,4,14.0,3,3,1919.0,206.0,11.0,3
top,학부,공과대학,경영학전공,YCD1602-01-00,통계학입문,3,김미정,"화4,목5,6",수영장(신촌),40,25,0 (N),0,0,0,0,O,36,1.0,36.0,1,1,36.0,N (N),6.0,N,Y,1.0,1.0,4.0,O
freq,124924,14695,5053,284,2062,51252,701,5280,1718,10056,2173,74354,121167,49716,48347,59153,74680,102263,61997.0,39216.0,1470,4006,12355.0,72373,54985.0,102537,118548,3384.0,19087.0,26749.0,112886


# Pandas Profiling 이용한 전처리 준비 (결과는 처참함...)

## 데이터 타입 변경 필요

In [36]:
pandas_profiling.ProfileReport(result)

Number of variables,32
Number of observations,124928
Total Missing (%),0.5%
Total size in memory,30.5 MiB
Average record size in memory,256.0 B
Numeric,1
Categorical,18
Boolean,0
Date,0
Text (Unique),0
Rejected,0


# 컬럼 하나씩 다 보면서 데이터 타입 변경 및 불필요 컬럼 제거

In [39]:
result.columns

Index(['구분1', '구분2', '구분3', '학정번호-분반-실습', '과목명', '학점', '담당교수', '강의시간', '강의실',
       '정원', '참여인원', '전공자 정원 (2전공포함)', '학년별정원 1학년', '학년별정원 2학년', '학년별정원 3학년',
       '학년별정원 4학년', '교환학생 가능여부', 'Max Mileage', '마일리지 최소값', '마일리지 최대값',
       '마일리지 평균값순위', '마일리지', '전공자/복수전공자(전공자정원포함여부)', '신청과목수', '졸업신청', '초수강여부',
       '총이수학점/졸업이수학점', '직전학기이수학점/학기당수강학점', '학년', '수강여부', '비고'],
      dtype='object')

# 1. '구분1' column --> 필요 없음.
## 이상치 파악 결과, 4번에 걸쳐 크롤링을 해서 header가 4번 들어갔음을 확인하였다
## 처음 불러올 때 해당 인덱스들 확인 후 삭제 요망

In [42]:
result['구분1'].unique()

array(['학부', '구분1'], dtype=object)

In [43]:
result['구분1'].describe()

count     124928
unique         2
top           학부
freq      124924
Name: 구분1, dtype: object

In [46]:
result[result['구분1'] == '구분1']

,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
33615,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
50281,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
34063,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
49587,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고


- 이상치 파악 결과, 4번에 걸쳐 크롤링을 해서 header가 4번 들어갔음을 확인하였다
- 처음 불러올 때 해당 인덱스들 확인 후 삭제 요망

In [48]:
data = result.drop([33615, 50281, 34063, 49587])

In [49]:
data['구분1'].unique()

array(['학부'], dtype=object)

# 2. '구분2' column = 학부대학
## data type = object --> category
- '공통기초(10-18)'은 '교양기초(2019~)'와 같음. 
- 단과대들은 놔두고, 나무지 전공이 아닌 것들은 따로 변수로 통합하면 된다

('교양기초(2019~)', '대학교양(2019~)', '기초교육(2019~)', '공통기초(10~18)','필수교양(10~18)', '선택교양(10~18)', '학부기초(~2009)', '학부필수(~2009)','계열기초(~2009)', '학부선택(~2009)','국제캠퍼스(2019~)', '(~2018)국제캠퍼스')

In [50]:
data['구분2'].unique()

array(['교양기초(2019~)', '대학교양(2019~)', '기초교육(2019~)', '공통기초(10~18)',
       '필수교양(10~18)', '선택교양(10~18)', '학부기초(~2009)', '학부필수(~2009)',
       '계열기초(~2009)', '학부선택(~2009)', '문과대학', '상경대학', '경영대학', '이과대학',
       '공과대학', '생명시스템대학', '신과대학', '사회과학대학', '음악대학', '생활과학대학', '교육과학대학',
       '언더우드국제대학', '약학대학', '글로벌인재대학', '연계전공', '국제캠퍼스(2019~)',
       '(~2018)국제캠퍼스', '교직과정', '의과대학', '치과대학', '평생교육사과정',
       'Study Abroad Course'], dtype=object)

안전빵으로 data2 만들어 object --> category 변환 시작

In [56]:
data2 = data.copy()

In [68]:
data2 = data2.astype({"구분2":'category'}) 

In [69]:
data2['구분2'].unique()

[교양기초(2019~), 대학교양(2019~), 기초교육(2019~), 공통기초(10~18), 필수교양(10~18), ..., 교직과정, 의과대학, 치과대학, 평생교육사과정, Study Abroad Course]
Length: 32
Categories (32, object): [교양기초(2019~), 대학교양(2019~), 기초교육(2019~), 공통기초(10~18), ..., 의과대학, 치과대학, 평생교육사과정, Study Abroad Course]

In [188]:
tt = combine_category(data2, '구분2', before, '교양')

In [185]:
before = ['교양기초(2019~)', '대학교양(2019~)', '기초교육(2019~)', '공통기초(10~18)','필수교양(10~18)', '선택교양(10~18)', '학부기초(~2009)', '학부필수(~2009)','계열기초(~2009)', '학부선택(~2009)','국제캠퍼스(2019~)', '(~2018)국제캠퍼스']

In [191]:
tt['구분2'].unique()

array(['교양', '문과대학', '상경대학', '경영대학', '이과대학', '공과대학', '생명시스템대학', '신과대학',
       '사회과학대학', '음악대학', '생활과학대학', '교육과학대학', '언더우드국제대학', '약학대학',
       '글로벌인재대학', '연계전공', '교직과정', '의과대학', '치과대학', '평생교육사과정',
       'Study Abroad Course'], dtype=object)

In [177]:
a = '채플'

In [182]:
l=[]
for i in range(12):
    l.append(a)

In [183]:
l

['채플', '채플', '채플', '채플', '채플', '채플', '채플', '채플', '채플', '채플', '채플', '채플']

In [131]:
test = data2.copy()

In [168]:
t1 = test['구분2'].replace(
    ['교양기초(2019~)', '대학교양(2019~)', '기초교육(2019~)', '공통기초(10~18)','필수교양(10~18)', '선택교양(10~18)', '학부기초(~2009)', '학부필수(~2009)','계열기초(~2009)', '학부선택(~2009)','국제캠퍼스(2019~)', '(~2018)국제캠퍼스'],
    ['교양','교양','교양','교양','교양','교양','교양','교양','교양','교양','교양','교양']
)

In [173]:
test['구분2'] = t1

In [174]:
test

,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
0,학부,교양,채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,1,2,Y (Y),6,N,Y,0.4603,0.8421,3,O
1,학부,교양,채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,2,2,Y (Y),5,Y,Y,0.873,0.8333,4,O
2,학부,교양,채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,3,1,Y (Y),6,N,Y,0.5238,0.6315,3,O
3,학부,교양,채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,4,1,Y (Y),5,N,Y,0.7857,0.5,4,O
4,학부,교양,채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,1,20,Y (Y),5,N,Y,0.2777,0.8947,2,O
5,학부,교양,채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,2,15,Y (Y),4,Y,Y,0.8769,0.8611,4,O
6,학부,교양,채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,3,5,N (N),6,N,Y,0.3214,0.9583,2,O
7,학부,교양,채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,4,4,Y (Y),6,N,Y,0.2698,0.8421,2,O
8,학부,교양,채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,5,4,Y (Y),5,N,Y,0.3412,1,2,O
9,학부,교양,채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,6,3,Y (Y),6,N,Y,0.5515,1,3,O


In [176]:
test['구분2'].unique()

array(['교양', '문과대학', '상경대학', '경영대학', '이과대학', '공과대학', '생명시스템대학', '신과대학',
       '사회과학대학', '음악대학', '생활과학대학', '교육과학대학', '언더우드국제대학', '약학대학',
       '글로벌인재대학', '연계전공', '교직과정', '의과대학', '치과대학', '평생교육사과정',
       'Study Abroad Course'], dtype=object)

In [162]:
c_list = ['교양기초(2019~)', '대학교양(2019~)', '기초교육(2019~)', '공통기초(10~18)','필수교양(10~18)', '선택교양(10~18)', '학부기초(~2009)', '학부필수(~2009)','계열기초(~2009)', '학부선택(~2009)','국제캠퍼스(2019~)', '(~2018)국제캠퍼스']
print(type(c_list))
len(c_list)

<class 'list'>


12

In [158]:
test.head()

,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
0,학부,교양기초(2019~),채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,1,2,Y (Y),6,N,Y,0.4603,0.8421,3,O
1,학부,교양기초(2019~),채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,2,2,Y (Y),5,Y,Y,0.873,0.8333,4,O
2,학부,교양기초(2019~),채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,3,1,Y (Y),6,N,Y,0.5238,0.6315,3,O
3,학부,교양기초(2019~),채플,YCA1001-01-00,채플(1),0,곽호철,화2,I종301,40,4,20 (N),2,18,15,5,O,36,1,2,1.5,4,1,Y (Y),5,N,Y,0.7857,0.5,4,O
4,학부,교양기초(2019~),채플,YCA1005-01-00,채플(A),0.5,곽호철,화2,I종301,646,61,500 (N),600,40,3,3,O,36,1,20,1.93,1,20,Y (Y),5,N,Y,0.2777,0.8947,2,O


In [149]:
'교양기초(2019~)' in c_list

True

In [148]:
test.loc[:,'구분2'] == '교양기초(2019~)'

0         True
1         True
2         True
3         True
4         True
5         True
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15        True
16        True
17        True
18        True
19        True
20        True
21        True
22        True
23        True
24        True
25        True
26        True
27        True
28        True
29        True
         ...  
61348    False
61349    False
61350    False
61351    False
61352    False
61353    False
61354    False
61355    False
61356    False
61357    False
61358    False
61359    False
61360    False
61361    False
61362    False
61363    False
61364    False
61365    False
61366    False
61367    False
61368    False
61369    False
61370    False
61371    False
61372    False
61373    False
61374    False
61375    False
61376    False
61377    False
Name: 구분2, Length: 124920, dtype: bool

# 3. '구분3' column = 전공
## data type object --> category

In [79]:
data2['구분3'].unique()

array(['채플', '기독교의이해', '글쓰기', '대학영어', '문학과예술', '인간과역사', '언어와표현', '가치와윤리',
       '국가와사회', '지역과세계', '논리와수리', '자연과우주', '생명과환경', '정보와기술', '체육과건강',
       '자율선택', '국가와사회공동체', '지역사회와세계', '소프트웨어', '역사·철학영역', '과학·기술영역',
       '사회·윤리영역', '인문·예술영역', '세계문화·언어영역', '생활·건강영역', '인간의이해', '자연의이해',
       '사회의이해', '문화의이해', '세계의이해', '계열기초', '인간이해와역사', '과학과기술', '사회와가치',
       '외국어와세계이해', '건강과스포츠', '문과대학 공통', '국어국문학전공', '사학전공', '철학전공',
       '문헌정보학전공', '심리학전공', '중어중문학전공', '영어영문학전공', '독어독문학전공', '불어불문학전공',
       '노어노문학전공', '상경대학 공통', '경제학전공', '응용통계학전공', '경영학전공', '이과대학 공통',
       '수학전공', '물리학전공', '화학전공', '지구시스템과학전공', '천문우주학전공', '대기과학전공',
       '공과대학공통', '화공생명공학전공', '전기전자공학전공', '건축공학전공', '건축학(설계)', '도시공학전공',
       '건설환경공학전공', '기계공학전공', '신소재공학전공', '산업공학전공', '컴퓨터과학전공', 'IT융합공학전공',
       '생명시스템공통', '시스템생물학전공', '생화학전공', '생명공학전공', '신학과', '사회과학대학공통',
       '정치외교학전공', '행정학전공', '사회복지학전공', '사회학전공', '문화인류학전공', '언론홍보영상학부',
       '음악대학공통', '교회음악전공', '성악전공', '피아노전공', '관현악전공', '작곡전공', '의류환경학전공',
       '식품영양학전공',

In [80]:
data2 = data2.astype({"구분3":'category'}) 

In [81]:
data2['구분3'].unique()

[채플, 기독교의이해, 글쓰기, 대학영어, 문학과예술, ..., GLD공통교과과정, 글로벌인재학부-바이오생활공학전공, 기초교육-자율선택, 간호(1학년), 선택교양-일반]
Length: 186
Categories (186, object): [채플, 기독교의이해, 글쓰기, 대학영어, ..., 글로벌인재학부-바이오생활공학전공, 기초교육-자율선택, 간호(1학년), 선택교양-일반]

In [106]:
len(data2['과목명'].unique())

2942

# 4. '학정번호-분반-실습' column  --> 제거
1. 수업을 구분하는  변수로 이미 '교과목명'이 있음
2. 동일 과목 별 다른 수업을 구분하나, 이미 교수 별로 나누어져 있고, 같은 교수의 동일한 두 과목이라도 시간으로 구분되어 있기 때문에 추가적인 정보는 제공하지 않는다고 판단.

<img src="reason1.png" width="1200">

In [86]:
data2['학정번호-분반-실습'].unique()

array(['YCA1001-01-00', 'YCA1005-01-00', 'YCA1005-02-00', ...,
       'IEE1154-02-00', 'IEE2083-01-00', 'IEE2086-01-00'], dtype=object)

In [100]:
data2['학정번호-분반-실습'].isnull().sum()

0

In [103]:
len(data2['학정번호-분반-실습'].unique())

3987

# 5. '과목명' column = 과목명
# 채플과 같이 과목명 다르지만 같은 수업들 선별해야함

In [87]:
data2['과목명'].unique()

array(['채플(1)', '채플(A)', '기독교와세계문화', ..., '직업진로설계', 'KOREAN ART HISTORY',
       'CORPORATE POLICY AND KOREAN ECONOMY'], dtype=object)

In [93]:
data2[data2['과목명'] == '채플']

,구분1,구분2,구분3,학정번호-분반-실습,과목명,학점,담당교수,강의시간,강의실,정원,참여인원,전공자 정원 (2전공포함),학년별정원 1학년,학년별정원 2학년,학년별정원 3학년,학년별정원 4학년,교환학생 가능여부,Max Mileage,마일리지 최소값,마일리지 최대값,마일리지 평균값순위,마일리지,전공자/복수전공자(전공자정원포함여부),신청과목수,졸업신청,초수강여부,총이수학점/졸업이수학점,직전학기이수학점/학기당수강학점,학년,수강여부,비고
